In [1]:
import os
import json
from typing import Dict, List, Optional

import litellm
from pydantic import BaseModel, Field, ValidationError
from dotenv import load_dotenv
import uuid
from enum import Enum

load_dotenv()

# Set your OpenAI API key (replace with your actual key)
litellm.api_key = os.getenv("OPENAI_API_KEY","")

class Movie(BaseModel):
    title: str
    director: str
    year: int

class Cinema(BaseModel):
    movies: List[Movie] = []

    class Config:
        extra = "forbid" #Prevent extra fields


In [2]:
# --- Movie Test ---
movie_schema = Cinema.model_json_schema(mode="validation")

prompt = f"""
Suggest a sci-fi movie from the 1990s and a comedy.  Return your response as a JSON object conforming to the following schema:

{json.dumps(movie_schema, indent=2)}
"""

# try:
#     response = litellm.completion(
#         model="gpt-3.5-turbo",  # Or another suitable model
#         messages=[{"role": "user", "content": prompt}],
#         response_format={"type": "json_object"},
#     )
#     movie_data = json.loads(response["choices"][0]["message"]["content"])
#     print("Movie Data:")
#     print(json.dumps(movie_data, indent=2))
#     Cinema.model_validate(movie_data) # Validate with Pydantic
#     print("Movie data validation successful!")
# except (json.JSONDecodeError, KeyError, IndexError) as e:
#     print(f"Error processing OpenAI response: {e}, Response: {response}")
# except ValidationError as e:
#     print(f"Validation error: {e}, Response: {response}")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")


In [ ]:
# --- TaskGraph Test ---
class NodeStatus(str, Enum):
    pending = "pending"
    in_progress = "in_progress"
    completed = "completed"
    failed = "failed"


class DependencyType(str, Enum):
    hard = "hard"
    soft = "soft"


class Edge(BaseModel):
    source: str
    target: str
    dependency_type: DependencyType = DependencyType.hard
    class Config:
        extra = "forbid"


class Node(BaseModel):
    id: str = Field(default_factory=lambda: str(uuid.uuid4()))
    description: str
    status: NodeStatus = NodeStatus.pending
    inputs: Optional[Dict] = None
    outputs: Optional[Dict] = None
    class Config:
        extra = "forbid"

class TaskGraph(BaseModel):
    nodes: Dict[str, Node] = {}
    edges: List[Edge] = []
    class Config:
        extra = "forbid"
    


# from graph import TaskGraph
#litellm._turn_on_debug()


graph_schema = TaskGraph.model_json_schema(mode="validation")

prompt = f"""
Create a task graph for baking a cake. The output MUST be valid JSON conforming to the schema below.

Schema: {graph_schema}
"""

try:
    response = litellm.completion(
        model="gpt-4o-2024-08-06",  # Or another suitable model
        messages=[{"role": "user", "content": prompt}],
        response_format=TaskGraph,
    )
    graph_data = json.loads(response["choices"][0]["message"]["content"])
    print("\nGraph Data:")
    print(json.dumps(graph_data, indent=2))
    TaskGraph.model_validate(graph_data)
    print("Graph data validation successful!")
except (json.JSONDecodeError, KeyError, IndexError) as e:
    print(f"Error processing OpenAI response: {e}, Response: {response}")
except ValidationError as e:
    print(f"Validation error: {e}, Response: {response}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

An unexpected error occurred: litellm.BadRequestError: OpenAIException - Invalid schema for response_format 'TaskGraph': In context=('properties', 'inputs', 'anyOf', '0'), 'additionalProperties' is required to be supplied and to be false.
